# Segmenting and Clustering Neighborhoods in Toronto

### Created by Justin Lee, 20 January 2020.


In this assignment, I will show how to explore, segment, and cluster the neighborhoods in the city of Toronto by scraping the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset demostrated in our tutorials.

Once the data is in a structured format, I will illustrate how to replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

## First Question (corresponds to the part of first submission):

1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.
3. To create the above dataframe:

    - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

    - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
    - Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    - In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4. **Submit a link to your Notebook on your Github repository. (10 marks)**


In [3]:
# import all necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

Firstly, I will use _BeautifulSoup4_ and _requests_ packages to extract the table content from Wikipedia. _requests_ is used to obtain the text content from the URL, while _BeautifulSoup_ is used to get the **wikitable sortable** table from the html text. By looping the rows in that table with a for-loop, all cells are read and their text contents are obtained as list objects, which is ready for _pandas_ dataframe creation.

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

url_text = requests.get(url).text
soup = BeautifulSoup(url_text,'lxml')
all_table = soup.find('table', class_='wikitable sortable')

Postcode_ = []
Borough_ = []
Neighbourhood_ = []
for row in all_table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) == 3:
        Postcode_.append(cells[0].find(text=True))
        Borough_.append(cells[1].find(text=True))
        Neighbourhood_.append(cells[2].find(text=True))

df = pd.DataFrame(Postcode_, columns = ['Postcode'])
df['Borough'] = Borough_
df['Neighbourhood'] = Neighbourhood_

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Here, we will clean the dataframe so that all **'Not assigned'** will be removed/amended, and the neighbourhoods of duplicated cells will be concatanated.

In [5]:
df = df[df.Borough != 'Not assigned']  # Ignore cells with a borough that is Not assigned.

for num, r in enumerate(df.iterrows()): # replace 'Not assigned' tags in "Neighbourhood" column with the corresponding borough.
    if r[1][2].find('Not assigned') >= 0:
        df.iloc[num]['Neighbourhood'] = r[1][1] 

df.Neighbourhood = df.Neighbourhood.str.strip()
df.Neighbourhood = df.Neighbourhood.str.replace('\n', '')
df.reset_index(drop=True, inplace=True)
# Merge the duplicated cells with same postcode.
Postcode_ = []
Borough_ = []
Neighbourhood_ = []
for postcode in df['Postcode'].unique():
    Postcode_.append(postcode)
    row_s = df[df.Postcode == postcode]
    if row_s.shape[0] == 1:
        Borough_.append(df[df.Postcode == postcode].Borough.to_string(index=False))
        Neighbourhood_.append(df[df.Postcode == postcode].Neighbourhood.to_string(index=False))
    else:
        new_Neighbourhood = df[df.Postcode == postcode].Neighbourhood.to_list()
        new_Borough = df[df.Postcode == postcode].Borough.to_list()
        Borough_.append(', '.join(set(new_Borough)))
        Neighbourhood_.append(', '.join(set(new_Neighbourhood)))

# We use the new dataframe to replace to old one.
df = pd.DataFrame(Postcode_, columns = ['Postcode'])
df['Borough'] = Borough_
df['Neighbourhood'] = Neighbourhood_

df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,Queen's Park


In [6]:
df.shape

(103, 3)

## Second Question (corresponds to the part of second submission):

In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

Differet from what we learnt from the older version of the tutorial, however, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

This Package may need your persistency sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

In [7]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

ModuleNotFoundError: No module named 'geocoder'

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:
<img src='https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/HZ3jNHNOEeiMwApe4i-fLg_f44f0f10ccfaf42fcbdba9813364e173_Screen-Shot-2018-06-18-at-7.18.16-PM.png?expiry=1579651200000&hmac=nUDFI8XvZHVCi1fqiKmLereahLhrZAfbPPq0vJLgfvg' height='400' width='400'>

Important Note: There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.

**Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. (2 marks)**

In [8]:
# Firstly, we read the online csv file to obtain the original dataframe.
import io
filename = ' http://cocl.us/Geospatial_data'
s = requests.get(filename).content
geo_df = pd.read_csv(io.StringIO(s.decode('utf-8')))
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
# Then, based on the postal code, we will merge borough and neighbourhood frome previous dataframe into this new dataframe.
complete_df = df.merge(geo_df, left_on='Postcode', right_on='Postal Code')
complete_df.drop('Postal Code', axis=1, inplace=True)
complete_df.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Third Question (corresponds to the part of third submission):

Finally, I will explore and cluster the neighborhoods in Toronto **(only boroughs that contain the word Toronto)**, and then replicate the same analysis we did to the New York City data. 

Remember:
- to add enough Markdown cells to explain what you decided to do and to report any observations you make.
- to generate maps to visualize your neighborhoods and how they cluster together. 

**Again, you should able to see this answer on the Github repository via a link. (3 marks)**

Firstly, we will import the necessary libraries; then, by using the _complete_df_ generated above, we will use **folium** to create our map with clustering.

In [10]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

Now, since we only consider Toronto data, we will filter out other unncessary rows and only keep the rows in which the borough contains _Toronto_ .

In [11]:
toronto_data = complete_df[complete_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Let's get the geographical coordinates of Toronto.

In [12]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [16]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [18]:
# The code was removed by Watson Studio for sharing.

Your credentails has been recorded
CLIENT_ID: *****
CLIENT_SECRET: *****


#### Let's explore the first neighborhood in our dataframe.

In [34]:
toronto_data.Neighbourhood = toronto_data.Neighbourhood.str.strip()
toronto_data.loc[0, 'Neighbourhood']

'Harbourfront'

In [35]:
neighbourhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


#### Now, let's get the top 100 venues that are in Harbourfront within a radius of 500 meters. We will use _requests_ to obtain information fromthe URL.

In [36]:
# type your answer here
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighbourhood_latitude, neighbourhood_longitude, VERSION, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?client_id=4BA0YEGXKCLPBAA1RUTMTIYZ5AB2SI0OGLRBE4ZBNCCS4RKD&client_secret=FGHXOCY0MBTWQZB22YTODGJEN1TRZXPCCY04TNBPLRDXLSKX&ll=43.6542599,-79.3606359&v=20180605&radius=500&limit=100'

In [37]:
results = requests.get(url).json()
if len(results) > 0:
    print('Resutls existed')

Resutls existed


Now the json data is ready to be cleaned and structure it into a **pandas** dataframe.

In [38]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [39]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [42]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

47 venues were returned by Foursquare.


## Explore Neighborhoods in Manhattan

Here is the function (defined in our lab) to repeat the same process of finding neighbourhoods in Toronto.

In [43]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [44]:
# type your answer here
LIMIT = 100
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'])
print(toronto_venues.shape)
toronto_venues.head()

Harbourfront
Queen's Park
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, Richmond, King
Dufferin, Dovercourt Village
Toronto Islands, Union Station, Harbourfront East
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Exhibition Place, Brockton, Parkdale Village
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill West, Forest Hill North
High Park, The Junction South
North Toronto West
North Midtown, Yorkville, The Annex
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Summerhill East, Moore Park
Chinatown, Grange Park, Kensington Market
South Hill, Forest Hill SE, Rathnelly, Summerhill West, Deer Park
CN Tower, Bathurst Quay, Island airport, Railway Lands, Harbourfront West, King and Spadina, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
St. James Town, Cabbagetown
Und

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Let's check how many venues were returned for each neighborhood

In [50]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues.groupby('Neighborhood').count().head()

There are 230 uniques categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, Richmond, King",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Railway Lands, Harbourfront West, King and Spadina, South Niagara",18,18,18,18,18,18
Central Bay Street,82,82,82,82,82,82


## Analyze Each Neighborhood

In [52]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
toronto_onehot.shape

(1705, 230)

In [56]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,"Adelaide, Richmond, King",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.00000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.00
2,Business Reply Mail Processing Centre 969 Eastern,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,"CN Tower, Bathurst Quay, Island airport, Railw...",0.000000,0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Central Bay Street,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,...,0.00000,0.000000,0.000000,0.00,0.012195,0.000000,0.000000,0.012195,0.000000,0.00
5,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00,0.045977,0.000000,0.057471,0.011494,0.000000,0.00
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Church and Wellesley,0.011628,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.011628,0.000000,0.011628,0.00
8,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.00000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.00
9,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.030303,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


Let's print out some neighborhood along with the top 5 most common venues.

In [59]:
num_top_venues = 5
n = 1
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    if n >= 5:
        break
    n += 1
    print('\n')

----Adelaide, Richmond, King----
         venue  freq
0  Coffee Shop  0.07
1   Steakhouse  0.04
2         Café  0.04
3          Bar  0.04
4   Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2              Bakery  0.04
3  Seafood Restaurant  0.04
4         Cheese Shop  0.04


----Business Reply Mail Processing Centre 969 Eastern----
           venue  freq
0    Yoga Studio  0.06
1  Garden Center  0.06
2    Pizza Place  0.06
3     Comic Shop  0.06
4     Restaurant  0.06


----CN Tower, Bathurst Quay, Island airport, Railway Lands, Harbourfront West, King and Spadina, South Niagara----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3       Coffee Shop  0.06
4               Bar  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1  Italian Restaurant  0.05
2                Café  0.04
3        Burger Joint  0.04
4      Ice 

#### Let's put that into a *pandas* dataframe and create a new dataframe to display the top 10 venues for each neighborhood.

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [63]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, Richmond, King",Coffee Shop,Steakhouse,Café,Bar,Thai Restaurant,Cosmetics Shop,Restaurant,Burger Joint,Hotel,Asian Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Beer Bar,Bakery,Steakhouse,Café,Farmers Market,Park
2,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Butcher,Restaurant,Burrito Place,Brewery,Light Rail Station
3,"CN Tower, Bathurst Quay, Island airport, Railw...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Bar,Plane
4,Central Bay Street,Coffee Shop,Italian Restaurant,Burger Joint,Sandwich Place,Café,Juice Bar,Japanese Restaurant,Ice Cream Shop,Salad Place,Bakery


## Cluster Neighborhoods

In [69]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2],
      dtype=int32)

### Let's make a dataframe to show the top-10 venues for each neighbourhood and visualize the clusters in a map.

In [72]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,2,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Café,Restaurant,Mexican Restaurant,Dessert Shop,Chocolate Shop
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,2,Coffee Shop,Park,Gym,College Auditorium,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place,Café,Portuguese Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Ramen Restaurant,Sporting Goods Shop,Italian Restaurant,Lingerie Store
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Restaurant,Café,Beer Bar,Breakfast Spot,Cosmetics Shop,Italian Restaurant,Bakery,Cocktail Bar,Hotel
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Other Great Outdoors,Pub,Trail,Health Food Store,Women's Store,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant


In [74]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters